In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import os
import pymysql
from tqdm import tqdm
import re
import time

In [40]:
ip_address = os.environ['AWS_IP']
PASSWORD = os.environ['SQL_PWD_AWS']

con = pymysql.connect(
    user='edward', 
    passwd=PASSWORD,
    host=ip_address, 
    db='community', 
    charset='utf8'
)

In [3]:
BASE_URL= 'https://www.fmkorea.com'

In [4]:
def get_urls(date):
    # %Y-%m-%d
    sql = f"select * from urls_fmk where date = '{date}'"
    with con.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
    return result

In [5]:
def is_dup_content(article_id):
    sql = f"select * from fmk_content where id = {article_id}"
    with  con.cursor() as cursor:
        cursor.execute(sql)
#     return bool(cursor.fetchall())
    return cursor.fetchall()

In [6]:
def commit_content(values):
    sql = f'insert into fmk_content(id, title, content, date) values {values}'
    with  con.cursor() as cursor:
        cursor.execute(sql)
        con.commit()

In [7]:
def commit_comment(values):
    sql = f'insert into fmk_comment(vote, content, article_id) values {values}'
    with  con.cursor() as cursor:
        cursor.execute(sql)
        con.commit()

In [ ]:
def only_hangul(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
    return hangul.sub('', s).strip()

In [8]:
urls = get_urls('2021-01-01')

In [9]:
headers = {'User-Agent': 'Mozilla/5.0'}

In [44]:
i = 1

In [45]:
for u in tqdm(urls[30:]):
    if i % 8 == 0:
        time.sleep(100)
    i += 1
    article_id, url_tail, date = u
    if is_dup_content(article_id):
        continue
    url = BASE_URL + url_tail
    result = requests.get(url, headers = headers)
    soup = BeautifulSoup(result.text, 'lxml')

    title_bar = soup.find('div', {'class': 'top_area'})
    title = title_bar.find('h1').text
    content = '\n'.join([x.text for x in soup.find('article').find_all('p') if x.text])

    values = (article_id, title, content ,date.strftime('%Y-%m-%d'))

    commit_content(values)

    comments = soup.find('ul', {'class': 'fdb_lst_ul '})
    comments = comments.find_all('li')
    for comment in comments:
        vote = comment.find('span', {'class': 'voted_count'}).text.strip()
        if not vote:  # 빈값이면
            vote = 0
        comment_content = comment.find('div', {'class': 'xe_content'})
        cc = comment_content.text.strip()
        if comment_content.find('a'):
            cc = ' '.join(cc.split()[1:])
        values = (vote, cc, article_id)
        try:
            commit_comment(values)
        except:
            values = (vote, only_hangul(values[1]), article_id)
            commit_comment(values)
    time.sleep(10)




  0%|          | 0/614 [00:00<?, ?it/s]


  0%|          | 2/614 [00:00<00:39, 15.60it/s]


  1%|          | 4/614 [00:14<22:21,  2.20s/it]


  1%|          | 5/614 [00:28<58:14,  5.74s/it]


  1%|          | 6/614 [00:43<1:25:05,  8.40s/it]


  1%|          | 7/614 [00:56<1:39:57,  9.88s/it]


  1%|▏         | 8/614 [02:54<7:07:39, 42.34s/it]


  1%|▏         | 9/614 [03:12<5:52:57, 35.00s/it]


  2%|▏         | 10/614 [03:29<4:57:42, 29.57s/it]


  2%|▏         | 11/614 [03:42<4:07:30, 24.63s/it]


  2%|▏         | 12/614 [03:57<3:37:16, 21.66s/it]


  2%|▏         | 13/614 [04:13<3:22:08, 20.18s/it]


  2%|▏         | 14/614 [04:27<3:02:47, 18.28s/it]


  2%|▏         | 15/614 [04:41<2:48:06, 16.84s/it]


  3%|▎         | 16/614 [06:33<7:32:58, 45.45s/it]


  3%|▎         | 17/614 [06:49<6:05:23, 36.72s/it]


  3%|▎         | 18/614 [07:02<4:54:08, 29.61s/it]


  3%|▎         | 19/614 [07:16<4:05:45, 24.78s/it]


  3%|▎         | 20/614 [07:30<3:35:01, 21.72s/it]


  3%|▎        

 22%|██▏       | 133/614 [59:04<2:57:28, 22.14s/it]


 22%|██▏       | 134/614 [59:17<2:36:53, 19.61s/it]


 22%|██▏       | 135/614 [59:39<2:42:20, 20.34s/it]


 22%|██▏       | 136/614 [1:01:42<6:45:01, 50.84s/it]


 22%|██▏       | 137/614 [1:02:00<5:27:34, 41.20s/it]


 22%|██▏       | 138/614 [1:02:20<4:35:33, 34.74s/it]


 23%|██▎       | 139/614 [1:02:33<3:44:15, 28.33s/it]


 23%|██▎       | 140/614 [1:02:52<3:20:45, 25.41s/it]


 23%|██▎       | 141/614 [1:03:14<3:12:23, 24.41s/it]


 23%|██▎       | 142/614 [1:03:28<2:48:31, 21.42s/it]


 23%|██▎       | 143/614 [1:03:49<2:45:41, 21.11s/it]


 23%|██▎       | 144/614 [1:05:45<6:29:45, 49.76s/it]


 24%|██▎       | 145/614 [1:06:31<6:19:04, 48.50s/it]


 24%|██▍       | 146/614 [1:07:23<6:25:40, 49.45s/it]


 24%|██▍       | 147/614 [1:07:45<5:22:03, 41.38s/it]


 24%|██▍       | 148/614 [1:08:24<5:15:05, 40.57s/it]


 24%|██▍       | 149/614 [1:09:36<6:28:10, 50.09s/it]


 24%|██▍       | 150/614 [1:10:36<6:50:17, 53.06s/it]


 42%|████▏     | 259/614 [2:01:55<2:32:30, 25.78s/it]


 42%|████▏     | 260/614 [2:02:10<2:14:23, 22.78s/it]


 43%|████▎     | 261/614 [2:02:23<1:56:52, 19.87s/it]


 43%|████▎     | 262/614 [2:02:43<1:56:02, 19.78s/it]


 43%|████▎     | 263/614 [2:02:54<1:40:59, 17.26s/it]


 43%|████▎     | 264/614 [2:04:47<4:27:47, 45.91s/it]


 43%|████▎     | 265/614 [2:05:04<3:36:22, 37.20s/it]


 43%|████▎     | 266/614 [2:05:21<3:01:12, 31.24s/it]


 43%|████▎     | 267/614 [2:05:40<2:38:04, 27.33s/it]


 44%|████▎     | 268/614 [2:05:55<2:17:11, 23.79s/it]


 44%|████▍     | 269/614 [2:06:12<2:05:12, 21.78s/it]


 44%|████▍     | 270/614 [2:06:23<1:46:33, 18.59s/it]


 44%|████▍     | 271/614 [2:06:38<1:39:39, 17.43s/it]


 44%|████▍     | 272/614 [2:08:34<4:27:13, 46.88s/it]


 44%|████▍     | 273/614 [2:08:48<3:30:45, 37.08s/it]


 45%|████▍     | 274/614 [2:09:06<2:57:47, 31.38s/it]


 45%|████▍     | 275/614 [2:09:22<2:31:45, 26.86s/it]


 45%|████▍     | 276/614 [2:09:34<2:05:27, 22.27

 63%|██████▎   | 385/614 [3:01:11<2:27:31, 38.65s/it]


 63%|██████▎   | 386/614 [3:01:22<1:55:23, 30.37s/it]


 63%|██████▎   | 387/614 [3:01:34<1:34:03, 24.86s/it]


 63%|██████▎   | 388/614 [3:01:54<1:27:39, 23.27s/it]


 63%|██████▎   | 389/614 [3:02:07<1:16:27, 20.39s/it]


 64%|██████▎   | 390/614 [3:02:19<1:07:00, 17.95s/it]


 64%|██████▎   | 391/614 [3:02:33<1:01:52, 16.65s/it]


 64%|██████▍   | 392/614 [3:04:31<2:53:29, 46.89s/it]


 64%|██████▍   | 393/614 [3:04:45<2:16:32, 37.07s/it]


 64%|██████▍   | 394/614 [3:04:57<1:48:41, 29.64s/it]


 64%|██████▍   | 395/614 [3:05:11<1:30:38, 24.83s/it]


 64%|██████▍   | 396/614 [3:05:23<1:17:03, 21.21s/it]


 65%|██████▍   | 397/614 [3:05:37<1:08:24, 18.92s/it]


 65%|██████▍   | 398/614 [3:05:52<1:04:14, 17.85s/it]


 65%|██████▍   | 399/614 [3:06:12<1:06:02, 18.43s/it]


 65%|██████▌   | 400/614 [3:08:06<2:47:34, 46.98s/it]


 65%|██████▌   | 401/614 [3:08:18<2:09:50, 36.57s/it]


 65%|██████▌   | 402/614 [3:08:30<1:43:31, 29.30

DataError: (1366, "Incorrect string value: '\\xF0\\x9F\\x91\\x8D\\xF0\\x9F...' for column 'content' at row 1")

In [160]:
con.close()